## Neural Network

### 1. Notations

- $L$ : 神经网络的层数。
- $n_l$ : 神经网络第 `l` 层的神经元个数。
- $\textbf{W}^{(l)} $: 从第 `l - 1`层到第 `l` 层权重矩阵, $\textbf{W}^{(l)} \in \mathbb{R}^{n_l \times n_{l - 1}}$。
- $w^{(l)}_{ij}$: 从第 `l - 1`层第 `j` 个神经元到 `l` 层第 `i` 个神经元的权重。
- $\textbf{b}^{(l)}$: 从第 `l - 1`层到第 `l` 层的偏置向量, $\textbf{b}^{(l)} \in \mathbb{R}^{n_l \times 1}$。
- $\textbf{z}^{(l)}$: 第 `l` 层的神经元状态，$\textbf{z}^{(l)} \in \mathbb{R}^{n_l \times 1}$。
- $\textbf{a}^{(l)}$: 第 `l` 层的神经元输出， $\textbf{a}^{(l)} \in \mathbb{R}^{n_l \times 1}$。
- $f(z^{(l)})$: 第 `l` 层的激活函数。

### 2. FP 公式

$$
\begin{align}
\textbf{z}^{(l)} &= \textbf{W}^{(l)} \textbf{a}^{(l-1)} + \textbf{b}^{(l)} \\
\textbf{a}^{(l)} &= f(\textbf{z}^{(l)})
\end{align}
$$

### 3. BP 公式

$$
\begin{align}
\delta^{L} &= -(\textbf{y} - \textbf{a}^{(L)}) \odot f'(\textbf{z}^{(l)}) \\
\delta^{l} &= \big((\textbf{W}^{(l+1)})^T \delta^{(l+1)} \big) \odot f'(\textbf{z}^{(l)}) \\
\frac{\partial{E}}{\partial{W^{l}}} &= \delta^{l}(\textbf{a}^{(l-1)})^{T} \\
\frac{\partial{E}}{\partial{b^{l}}} &= \delta^{l}
\end{align}
$$

### 4. BP 批量梯度实现

『批量梯度下降』算法更新参数总结如下：
1. 用 BP 算法四个公式**求得每一个训练数据**的代价函数对参数的偏导数：
2. 按下面公式更新参数：

$$
\begin{align}
\textbf{W}^{(l)} = \textbf{W}^{(l)} - \frac{\mu}{N}\sum_{i=1}^{N} \frac{\partial{E_{(i)}}}{\partial{W^{l}}} \\
\textbf{b}^{(l)} = \textbf{b}^{(l)} - \frac{\mu}{N}\sum_{i=1}^{N} \frac{\partial{E_{(i)}}}{\partial{b^{l}}}
\end{align}
$$

3. 迭代执行 (1),(2) 步，直到满足停止准则(比如相邻两次迭代的差别很小，或者直接限制迭代次数）

**注意**，数据量大时，可以使用 mini-batch 来更新，也可以每次仅使用一个训练数据来更新参数。（公使用一个训练参数来更新，是支持在线训练的）

### 5. 实现细节

#### 5.1 参数初始化

对于每一层的权重参数 $W^{l} \in R^{n_l \times n_{l - 1}}$  在进行初始化时需要注意以下知识点：

1. 全部默认初始化为 0

这样会造成 NN 和普通的线性模型没有什么不同。

2. 随机初始化

如使用标准分布进行随机初始化: $w_l = np.random.randn(nl, nl_1)$，这样会造成两个问题：

    - 梯度消失 (Vanishing gradients)

    当初始化的值很小是地，在 BP 时 $dw$ 变得越来越小。特别如果使用 Sigmoid 激活函数会变得更小。**梯度消失** 会造成收敛变慢，甚至会停止训练。

    - 梯度爆炸 (Exploding gradients)

    当初始化的值很大时，在 FP 时也会传播，造成损失变大。梯度爆炸会造成函数收敛振荡（oscillating）, 还很可能会错过最优值，不会收敛。而且还可能在计算中出现 NaN 值出现。 
    


3. **最佳实践**

    - 选择合适的激活函数

        如可以选择 ReLU 或 Leaky ReLU

    - 使用启发式的初始化函数

        1. 对于 ReLU 激活函数可以使用， 也称为 **He initialization**:
    
    ```
    wl = np.random.randn(nl, nl_l) * np.sqrt(2 / nl_1)
    ```
    
        即参数用正态分布后乘上 $\sqrt{\frac{2}{nl\_1}}$
    
        2. 对于 tanh 激活函数可以使用, 也称为 **Xavier initialization**, 和上面的区别就是 2 变成了 1：
       
    ```
    wl = np.random.randn(nl, nl_l) * np.sqrt(1 / nl_1)
    ```
    
         即参数用正态分布后乘上 $\sqrt{\frac{1}{nl\_1}}$
     
        3. 另外一种启发式初始化是使用正态分成乘上 $\sqrt{\frac{1}{nl + nl\_1}}$
     
     ```
     wl = np.random.randn(nl, nl_l) * np.sqrt(1 / (nl + nl_1))
     ```

4. 梯度裁剪（Gradient Clipping)

    如果当参数超过某个阈值后就将其裁剪。


#### 5.2 激活函数

在参数初始化时，我们也提到到不同一的激活函数的影响。常见的隐藏层的激活函数有：

1. Sigmoid 激活函数

$$
f(x) = \sigma(x) = \frac{1}{1 + e^{-x}}
$$

    - 值域：[0, 1]
    - 特点：原函数单调，但导数不单调。
    - 缺点：梯度容易消失。导致收敛缓慢或不收敛。
    
2. Tanh 激活函数

$$
f(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}
$$

    - 值域：[-1, 1]
    - 特点：原函数和导数都单调(monotonic)， Output is zero centered, 容易优化。主要用于二分类问题中。
    - 特点：也有梯度消失的问题，导致不会收敛。
    
3. ReLu 

$$
f(x) = max(0, x) 
$$
    - 值域: (0, +Inf)
    - 特点：原函数和导数都单调(monotonic)，容易计算
    - 缺点：Output is not zero centered. 当取值为负时，梯度为 0, 不会收敛到最优结果 。

4. Leaky ReLu

$$
f(x) = max(\alpha x, x)
$$

\alpha = 0.01 称之为 Leaky Relu, 而不是 `0.01` 称之为 Randomized Relu.

    - 值域: [-Inf, +Inf]
    - 特点：解决了 Relu 的取值为负时，梯度为 0 的问题。


而非激活层的激活函数，最常见的就是 Softmax 了。

$$
f(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{K} e^{x_j}}, i = 1, \dots K
$$

### 5.3 梯度计算优化方案


优化的基础 **EWMA (Exponentially Weighted Moving Average)**，指数权重移动平均线，均线。

1. Momentum, 因为 mini-batch 的梯度并不一定正确只是一个估计，使用 EWMA 的方法来做估计，修正 Gradient, 提高收敛速度。 引用 β 超参.
2. AdaGrad (Adaptive Learning Rate) 根据 Gradient 的信息来调整学习因子。
3. AdaDelta, RMSProp 都是根据 Gradient 的信息来调整学习因子，但是是使用 EWMA 的方法来调整。
4. Adam, 是将 RMSProp 与 Momentum 接合起来。

#### 5.1  指数移动平均(Exponential moving average, EMA 或 EXMA) 

EMA 是以指数式递减加权的移动平均。各数值的加权影响力随时间而呈指数式递减，近期的数据加权影响力越重，但较旧的数据也给予一定的加权值。

加权的程度以常数 $\alpha \in (0, 1)$ 决定，$\alpha$ 也可以用天数 N 来代表: $\alpha = \frac{2}{N + 1}, N = 19天$, 则 $\alpha = 0.1$。

设时间 $t$ 的实际数值为 $Y_t$, 其时间 t 的 EMA 为 $S_t$, 则有：

$$
S_t = \alpha \times Y_t + (1 - \alpha) \times S_{t-1}
$$

### 5.2 Momentum (or SGD with Momentum)

Momentum 也是依据 EMA 的思想。因为在使用 mini-batch 算法时，计算出的的梯度并不一定正确，只是一个估计。那么使用 EMA 的思想，来对 Gradient 做修正，提高梯度的正确性，提高收敛速度。这里引入了一个 $\beta$ 参数用于作为 EMA 的参数。

更新公式为：

$$
V_t = \beta V_{t-1} + (1 - \beta) \nabla{w}L(W, X, y) \\
W = W - \alpha V_t
$$

更一般的写法为：

$$
V_t = \beta V_{t-1} + \alpha \nabla{w}L(W, X, y) \\
W = W -  V_t
$$

#### Nesterov Momentum (涅斯捷罗夫动量)

Nesterov Momentum 与普通的动量有一点不同。在这个版本中，我们先根据当前的梯度向前看一步 （lookahead).

其更新公式为：

$$
V_t = \beta V_{t-1} + \alpha \nabla{w}L(W - \beta V_{t-1}, X, y) \\
W = W -  V_t
$$

<img src="http://cs231n.github.io/assets/nn3/nesterov.jpeg" />


### 6. 程序流程

1. 参数初始化
2. 激活函数
3. 激活函数导数
4. evaluate(test_data)
5. FP 
6. model(train_data, epoches, alpha, mini_batch_size, test_data)
7. update_mini_batch()
8. BPm

## 神经网络的优化方法
1. 参数初始化
2. 激活函数梯度消失
3. dropout


#### References

1. [Deep Learning Best Practices (1) — Weight Initialization](https://medium.com/usf-msds/deep-learning-best-practices-1-weight-initialization-14e5c0295b94)
2. [wiki: Activation function ](https://en.wikipedia.org/wiki/Activation_function)
3. [Topic DL01: Activation functions and its Types in Artifical Neural network](https://medium.com/@abhigoku10/activation-functions-and-its-types-in-artifical-neural-network-14511f3080a8)
4. [指数移动平均](https://zh.wikipedia.org/wiki/%E7%A7%BB%E5%8B%95%E5%B9%B3%E5%9D%87)